In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
import math

In [2]:
albums = []
albumFile = open('/dbfs/FileStore/tables/dataset_7k_2010-f3f46.txt', 'r')
for line in albumFile:
  albums.append(line.rstrip().split('\t'))
  for x in range(len(albums[-1])):
    if x > 3:
      albums[-1][x] = float(albums[-1][x])
  albums[-1] = [albums[-1][0], albums[-1][1], albums[-1][2], albums[-1][3], [f for f in albums[-1][4:]]]
albumFile.close()

  
albumSchema = StructType([ \
    StructField("AI", StringType()), \
    StructField("AN", StringType()), \
    StructField("TI", StringType()), \
    StructField("TN", StringType()), \
    StructField("FTS", ArrayType(DoubleType()))])

albumDF = sqlContext.createDataFrame(albums, albumSchema)

In [3]:
albumDF.groupBy('AI').agg(collect_list('FTS')).take(1)

Out[ 36 ]: [Row(AI='0rymLJxRADVhIDmjLtVvhF', collect_list(FTS)=[[0.849, 0.367, 250280.0, 0.21, 1e-06, 7.0, 0.34, -19.451, 1.0, 0.432, 102.225, 3.0, 0.0868], [0.877, 0.191, 214800.0, 0.381, 0.923, 9.0, 0.313, -12.811, 1.0, 0.0377, 136.048, 4.0, 0.194], [0.836, 0.678, 38253.0, 0.295, 0.0, 6.0, 0.884, -17.144, 0.0, 0.909, 86.022, 4.0, 0.691], [0.737, 0.71, 112467.0, 0.705, 0.0, 0.0, 0.089, -8.888, 1.0, 0.105, 128.786, 4.0, 0.855], [0.735, 0.572, 233880.0, 0.389, 0.0, 0.0, 0.0686, -13.24, 1.0, 0.449, 139.03, 4.0, 0.721], [0.911, 0.101, 103960.0, 0.0777, 0.702, 3.0, 0.104, -17.819, 1.0, 0.0429, 75.192, 3.0, 0.0381], [0.806, 0.235, 259680.0, 0.277, 0.0, 5.0, 0.128, -14.638, 1.0, 0.0456, 86.794, 4.0, 0.0743], [0.808, 0.487, 273067.0, 0.27, 0.0, 3.0, 0.447, -12.794, 1.0, 0.161, 137.055, 4.0, 0.248], [0.74, 0.471, 148280.0, 0.605, 0.0, 1.0, 0.78, -14.201, 1.0, 0.739, 95.709, 3.0, 0.366], [0.799, 0.498, 109627.0, 0.367, 0.0, 5.0, 0.298, -15.641, 0.0, 0.623, 96.728, 4.0, 0.473], [0.855, 0.652, 148680.0, 0.2, 0.0, 7.0, 0.101, -15.996, 1.0, 0.594, 85.782, 4.0, 0.306], [0.948, 0.235, 272973.0, 0.0234, 8e-06, 4.0, 0.167, -26.22, 1.0, 0.047, 81.675, 5.0, 0.107], [0.907, 0.298, 453360.0, 0.286, 9e-06, 4.0, 0.22, -13.759, 1.0, 0.0452, 89.377, 4.0, 0.155], [0.96, 0.311, 170653.0, 0.202, 1.1e-05, 6.0, 0.0972, -15.323, 0.0, 0.0464, 82.225, 3.0, 0.057], [0.735, 0.496, 75853.0, 0.0373, 1e-06, 7.0, 0.126, -22.535, 1.0, 0.064, 117.57, 3.0, 0.0834], [0.799, 0.5, 372040.0, 0.222, 0.0, 7.0, 0.274, -14.844, 1.0, 0.462, 80.926, 4.0, 0.428], [0.902, 0.322, 212213.0, 0.0543, 1.1e-05, 10.0, 0.746, -17.857, 1.0, 0.0382, 88.013, 3.0, 0.0516], [0.00882, 0.596, 275613.0, 0.633, 0.0, 0.0, 0.0564, -8.008, 1.0, 0.0446, 117.015, 4.0, 0.277], [0.763, 0.28, 442867.0, 0.273, 4e-06, 7.0, 0.0636, -12.323, 0.0, 0.0447, 89.942, 3.0, 0.0704], [0.853, 0.221, 197280.0, 0.384, 0.108, 2.0, 0.0733, -12.181, 1.0, 0.0318, 79.579, 3.0, 0.242], [0.893, 0.32, 398880.0, 0.054, 9e-06, 0.0, 0.129, -22.449, 0.0, 0.0491, 80.116, 3.0, 0.106], [0.565, 0.568, 231933.0, 0.316, 0.0, 5.0, 0.593, -13.604, 0.0, 0.567, 98.635, 3.0, 0.579], [0.801, 0.496, 147973.0, 0.676, 0.0, 0.0, 0.651, -11.041, 1.0, 0.343, 138.726, 4.0, 0.543], [0.849, 0.485, 83427.0, 0.436, 5.8e-05, 7.0, 0.448, -11.204, 1.0, 0.0522, 67.287, 3.0, 0.361], [0.794, 0.522, 243653.0, 0.349, 0.0, 7.0, 0.692, -11.879, 1.0, 0.274, 144.793, 4.0, 0.433], [0.82, 0.635, 75973.0, 0.327, 0.0, 11.0, 0.305, -15.154, 1.0, 0.48, 135.953, 5.0, 0.724], [0.888, 0.254, 571960.0, 0.127, 0.000141, 5.0, 0.191, -14.697, 1.0, 0.0422, 82.63, 4.0, 0.0772], [0.86, 0.462, 175213.0, 0.23, 1.5e-05, 5.0, 0.0841, -13.95, 0.0, 0.0423, 94.708, 3.0, 0.296], [0.938, 0.191, 370667.0, 0.17, 0.00123, 10.0, 0.103, -13.883, 1.0, 0.0431, 79.518, 4.0, 0.0581], [0.863, 0.405, 149800.0, 0.123, 0.0, 6.0, 0.182, -16.394, 1.0, 0.0678, 78.862, 3.0, 0.0586], [0.715, 0.422, 298600.0, 0.337, 0.0, 5.0, 0.406, -13.556, 0.0, 0.399, 165.68, 5.0, 0.197], [0.899, 0.235, 861720.0, 0.0613, 8e-06, 5.0, 0.0615, -19.13, 1.0, 0.0551, 77.223, 4.0, 0.0436]])]

In [4]:
mergedAlbumDF = albumDF.groupBy('AI').agg(collect_list('FTS').alias('FTS'))

In [5]:
def interpolateRow(row):
  finalRow = [row.AI, [ [ np.nan for _ in range(13) ] for _ in range(24) ]]
  
  # For each feature type
  for i in range(13):
    
    tempFeatures = []
    
    # Divide case to albums with single track and multiple tracks
    # to avoid 'Division by zero' exception
    # If multiple tracks
    if(len(row.FTS) != 1):
      interSpace = math.floor(((24 - len(row.FTS)) / (len(row.FTS) - 1)))
      additionalSpace = (24 - len(row.FTS)) % (len(row.FTS) - 1)
    
      ## Value and NaN placements
      # For each list of track features
      for j in row.FTS:
        tempFeatures.append(j[i])
        for s in range(interSpace):
          tempFeatures.append(np.nan)
        if additionalSpace > 0:
          tempFeatures.append(np.nan)
          additionalSpace -= 1
    
    # If single track
    else:
      tempFeatures.append(row.FTS[0][i])
      for n in range(23):
        tempFeatures.append(np.nan)
    
    ## Interpolate
    tempFeatures = pd.Series(tempFeatures)
    tempFeatures = tempFeatures.interpolate()
    
    ## Assign Back
    for j in range(24):
      finalRow[1][j][i] = tempFeatures[j]
  
  return finalRow


finalAlbums = mergedAlbumDF.filter(size(mergedAlbumDF.FTS) <= 24).rdd.map(interpolateRow).cache()

In [6]:
def stringify(data):
  row = data[0]
  for x in data[1]:
    for y in x:
      row += '\t%f' % y
  return row

In [7]:
%fs rm -r dbfs:/FileStore/tables/2010.tsv

res4: Boolean = false

In [8]:
# Save the RDD
finalAlbums.map(stringify).saveAsTextFile('dbfs:/FileStore/tables/2010.tsv')

In [9]:
# Check Output File
sc.textFile('dbfs:/FileStore/tables/2010.tsv').take(3)

Out[ 54 ]: 
['4HOobzcnXNTNaKxP2MIBrg\t0.009060\t0.386000\t288960.000000\t0.883000\t0.000158\t2.000000\t0.648000\t-5.155000\t0.000000\t0.074400\t98.630000\t4.000000\t0.150000\t0.006535\t0.325500\t257600.000000\t0.929000\t0.000080\t2.000000\t0.446000\t-4.244500\t0.000000\t0.168700\t136.341000\t4.000000\t0.319000\t0.004010\t0.265000\t226240.000000\t0.975000\t0.000002\t2.000000\t0.244000\t-3.334000\t0.000000\t0.263000\t174.052000\t4.000000\t0.488000\t0.002376\t0.292000\t235520.000000\t0.944500\t0.001451\t2.000000\t0.453500\t-3.979000\t0.000000\t0.171350\t180.069500\t4.000000\t0.331500\t0.000741\t0.319000\t244800.000000\t0.914000\t0.002900\t2.000000\t0.663000\t-4.624000\t0.000000\t0.079700\t186.087000\t4.000000\t0.175000\t0.000567\t0.307500\t258860.000000\t0.925000\t0.001494\t6.000000\t0.504000\t-4.509500\t0.500000\t0.087950\t142.861000\t4.000000\t0.322000\t0.000394\t0.296000\t272920.000000\t0.936000\t0.000089\t10.000000\t0.345000\t-4.395000\t1.000000\t0.096200\t99.635000\t4.000000\t0.469000\t0.005547\t0.327500\t305653.500000\t0.909000\t0.000406\t6.000000\t0.510000\t-4.604500\t0.500000\t0.074050\t102.608000\t4.000000\t0.395500\t0.010700\t0.359000\t338387.000000\t0.882000\t0.000723\t2.000000\t0.675000\t-4.814000\t0.000000\t0.051900\t105.581000\t4.000000\t0.322000\t0.006095\t0.323000\t288113.500000\t0.924500\t0.000365\t2.000000\t0.679500\t-3.986500\t0.000000\t0.110450\t146.896000\t4.000000\t0.344500\t0.001490\t0.287000\t237840.000000\t0.967000\t0.000008\t2.000000\t0.684000\t-3.159000\t0.000000\t0.169000\t188.211000\t4.000000\t0.367000\t0.166245\t0.381000\t142080.000000\t0.703000\t0.008404\t4.500000\t0.426000\t-8.685000\t0.500000\t0.115300\t129.094000\t4.000000\t0.430500\t0.331000\t0.475000\t46320.000000\t0.439000\t0.016800\t7.000000\t0.168000\t-14.211000\t1.000000\t0.061600\t69.977000\t4.000000\t0.494000\t0.165963\t0.382500\t138746.500000\t0.701000\t0.008403\t9.000000\t0.133400\t-9.108500\t0.500000\t0.078800\t124.818000\t4.000000\t0.363500\t0.000925\t0.290000\t231173.000000\t0.963000\t0.000007\t11.000000\t0.098800\t-4.006000\t0.000000\t0.096000\t179.659000\t4.000000\t0.233000\t0.001623\t0.320500\t218080.000000\t0.968000\t0.000005\t11.000000\t0.414400\t-3.946500\t0.000000\t0.199500\t189.776000\t4.000000\t0.193500\t0.002320\t0.351000\t204987.000000\t0.973000\t0.000003\t11.000000\t0.730000\t-3.887000\t0.000000\t0.303000\t199.893000\t4.000000\t0.154000\t0.002330\t0.359000\t213107.000000\t0.971000\t0.000003\t5.500000\t0.486500\t-4.219000\t0.500000\t0.292500\t190.005000\t4.000000\t0.232000\t0.002340\t0.367000\t221227.000000\t0.969000\t0.000004\t0.000000\t0.243000\t-4.551000\t1.000000\t0.282000\t180.117000\t4.000000\t0.310000\t0.002800\t0.433000\t205647.000000\t0.960000\t0.000002\t1.000000\t0.279500\t-4.238500\t0.500000\t0.175150\t134.547000\t4.000000\t0.473500\t0.003260\t0.499000\t190067.000000\t0.951000\t0.000000\t2.000000\t0.316000\t-3.926000\t0.000000\t0.068300\t88.977000\t4.000000\t0.637000\t0.352630\t0.546500\t125980.000000\t0.629500\t0.246000\t5.500000\t0.222000\t-10.334000\t0.000000\t0.064100\t94.501500\t3.500000\t0.461000\t0.702000\t0.594000\t61893.000000\t0.308000\t0.492000\t9.000000\t0.128000\t-16.742000\t0.000000\t0.059900\t100.026000\t3.000000\t0.285000\t0.001450\t0.322000\t444653.000000\t0.954000\t0.000337\t0.000000\t0.407000\t-5.657000\t1.000000\t0.162000\t140.059000\t3.000000\t0.259000',
 '0lg6bgtAuwJDfPnGYrf5Do\t0.001200\t0.559000\t365560.000000\t0.885000\t0.000620\t4.000000\t0.132000\t-5.969000\t0.000000\t0.069300\t124.461000\t4.000000\t0.319000\t0.000659\t0.521500\t261800.000000\t0.935500\t0.000389\t2.000000\t0.146500\t-5.020000\t0.500000\t0.100650\t127.628000\t4.000000\t0.364500\t0.000118\t0.484000\t158040.000000\t0.986000\t0.000158\t0.000000\t0.161000\t-4.071000\t1.000000\t0.132000\t130.795000\t4.000000\t0.410000\t0.000305\t0.523000\t245566.500000\t0.907500\t0.002869\t2.000000\t0.162000\t-4.891000\t0.500000\t0.083050\t133.513000\t4.000000\t0.363500\t0.000492\t0.562000\t333093.000000\t0.829000\t0.005580\t4.000000\t0.1630